In [1]:
import folium
import pandas as pd
import time
import io
import os

from joblib import Parallel, delayed

from PIL import Image, ImageDraw, ImageFont


In [2]:
m = folium.Map(location=[38.919,-77.013], zoom_start=12) #generation of map

folium.CircleMarker([38.919,-77.013], radius=12, tooltip='DC', fill=True).add_to(m) # circle generation

m.save('map.html')
m

In [4]:
#API
#datos = pd.read_csv('../scraper/datos/bigdatos.csv')
#datos.head()

In [ ]:
import request


In [3]:
datos_ozono = datos[datos['ParameterName'] == 'OZONE']
datos_PM2_5 = datos[datos['ParameterName'] == 'PM2.5']
datos_PM10 = datos[datos['ParameterName'] == 'PM10']

# VISUALIZACIÓN OZONO 

In [4]:
datos_ozono.head()

,DateObserved,HourObserved,LocalTimeZone,ReportingArea,StateCode,Latitude,Longitude,ParameterName,AQI,CategoryNumber,CategoryName
0,1/1/2018,0,EST,Metropolitan Washington,DC,38.919,-77.013,OZONE,33,1,Good
3,1/2/2018,0,EST,Metropolitan Washington,DC,38.919,-77.013,OZONE,15,1,Good
6,1/3/2018,0,EST,Metropolitan Washington,DC,38.919,-77.013,OZONE,15,1,Good
9,1/4/2018,0,EST,Metropolitan Washington,DC,38.919,-77.013,OZONE,26,1,Good
12,1/5/2018,0,EST,Metropolitan Washington,DC,38.919,-77.013,OZONE,17,1,Good


In [5]:
ozono = folium.Map(location=[38.919,-77.013], zoom_start=12) #generation of map

folium.CircleMarker([38.919,-77.013], radius=33, tooltip='DC', fill=True).add_to(ozono) # circle generation

ozono

In [6]:
ozono = folium.Map(location=[38.919,-77.013], zoom_start=12) #generation of map

folium.CircleMarker([38.919,-77.013], radius=15, tooltip='DC', fill=True).add_to(ozono) # circle generation

ozono

In [39]:
def plot_map(df_medicion, zoom_start=20):
    
    
    for i, row in df_medicion.iterrows():
        if row.CategoryName == 'Good':
            color="#008000" #GREEN
            
        elif row.CategoryName == 'Moderate':
            color="#FFFF00" #YELLOW
        
        elif row.CategoryName == 'Unhealthy': 
            color="FF8000" #ORANGE
            
        elif row.CategoryName == 'Unhealthy for Sensitive Groups':
            color="#B22222" #RED
            
        folium_map = folium.Map(location=[38.919,-77.013],
                            tiles='cartodb positron', zoom_start=zoom_start)

    
        folium.CircleMarker(location=[row.Latitude, row.Longitude], radius=row.AQI/2, color=color,
                                fill=True, fill_opacity=0.4).add_to(folium_map)

        folium_map.save('{}.html'.format(i))

    return folium_map

In [40]:
plot_map(datos_ozono)

In [54]:
def plot_map(i, row, zoom_start=20):
    
    if row.CategoryName == 'Good':
            color="#008000" #GREEN
            
    elif row.CategoryName == 'Moderate':
            color="#FFFF00" #YELLOW
        
    elif row.CategoryName == 'Unhealthy': 
            color="FF8000" #ORANGE
            
    elif row.CategoryName == 'Unhealthy for Sensitive Groups':
            color="#B22222" #RED

    folium_map = folium.Map(location=[38.919,-77.013],
                        tiles='cartodb positron', zoom_start=zoom_start)


    folium.CircleMarker(location=[row.Latitude, row.Longitude], radius=row.AQI/2, color=color,
                            fill=True, fill_opacity=0.4).add_to(folium_map)

    #folium_map.save('{}.html'.format(i))

    png = folium_map._to_png()
    time.sleep(1.5)
    image = Image.open(io.BytesIO(png))
    draw = ImageDraw.ImageDraw(image)
    draw.text((20,image.height - 50), 
              "date:{}".format(row.DateObserved),
              fill=(0, 0, 0))

    filename = os.path.join(os.listdir('.'), "frame_{:0>5}.png".format(i))
    image.save(filename, "PNG")


In [ ]:
Parallel(n_jobs=12, verbose=10)(delayed (plot_map)(i, row, zoom_start=20) for i, row in datos_ozono.iterrows())

In [ ]:
import imageio
filenames = sorted((fn for fn in os.listdir('.') if fn.endswith('.png')))
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('datos.gif', images, fps=5)